## Design Thawing Index
This notebook provides a simple computation of the design thawing index by averaging the three warmest (greatest thawing index values) years from a given range of years.
Users can modify the number of warmest years used to compute the average, and the range of years for which the design thawing index value will be computed.

### Introduction to Jupyter Notebooks

[Jupyter Notebooks](https://jupyter.org/) are a popular tool for interactive computing used by data professionals. Notebooks provide a versatile environment for writing and executing code, creating visualizations, and documenting your work.

#### Running Individual Cells

Jupyter Notebooks are organized into cells. Each cell can contain code or text. To run an individual code cell:

1. Click on the cell you want to run. The cell will be highlighted.
2. Press `Shift + Enter` on your keyboard, or click the "Run" button in the toolbar at the top. The code in the cell will execute, and any output (such as printed text or graphs) will be displayed immediately below the cell.
3. You can run cells in any order, making it easy to experiment and iterate with your code. Note that the notebook here expects cells to be executed in order to produce the intended results.

#### Running All Cells in Sequence

To run all cells in the notebook in sequence:

1. Go to the "Cell" menu at the top.
2. Select "Run All." This will execute all the cells in the notebook from top to bottom.
3. Running all cells is useful when you want to ensure that your notebook runs correctly from start to finish.

#### Downloading a Jupyter Notebook

When you want to save a local copy of the notebook:

1. Go to the "File" menu.
2. Select "Download" - this will save your notebook in the native Jupyter Notebook format (**.ipynb**). and the notebook will be downloaded to your computer.


In [ ]:
# edit and execute this cell to choose a latitude and longitude (decimal degrees) for your computation
latitude = 65.1
longitude = -146.6

In [ ]:
# this cell helps run Python code in your browser
import micropip
await micropip.install('pyodide-http')
import pyodide_http
pyodide_http.patch_all()

In [ ]:
# this cell uses the SNAP Data API to fetch a thawing index table
# results include different climate models
import pandas as pd
df = pd.read_csv(f"https://earthmaps.io/degree_days/thawing_index/{latitude}/{longitude}?format=csv", header=3)
df

In [ ]:
# function to compute design thawing index
def compute_and_display_design_index(start_year, end_year, n_warmest_years):
    df_slice = df[(df["year"] >= start_year) & (df["year"] < end_year)]
    design_index = round(df_slice.sort_values("dd", ascending=False).iloc[0:n_warmest_years]["dd"].mean())
    return design_index

In [ ]:
# this cell enables the use of interactive sliders to control the inputs for computation
await micropip.install("ipywidgets")
import ipywidgets as widgets
from IPython.display import display

start_year_slider = widgets.IntSlider(
    value=2030, min=2018, max=2098, step=1, description="Start Year:"
)
end_year_slider = widgets.IntSlider(
    value=2060, min=2019, max=2099, step=1, description="End Year:"
)
n_coldest_years_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1, description="Number of Warmest Years:",
    style={"description_width": "initial"}
)

# create an output widget for dynamic displaying of result
output = widgets.Output()

# function to handle widget changes
def handle_slider_change(change):
    with output:
        output.clear_output()

        if (
            end_year_slider.value - start_year_slider.value
            < n_coldest_years_slider.value
        ):
            start_year_slider.value = (
                end_year_slider.value - n_coldest_years_slider.value
            )
            output.clear_output()
        result = compute_and_display_design_index(
            start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
        )
        print(f"Design Thawing Index: {result} °F•days")

# attach the change handler to the sliders
start_year_slider.observe(handle_slider_change, names="value")
end_year_slider.observe(handle_slider_change, names="value")
n_coldest_years_slider.observe(handle_slider_change, names="value")

handle_slider_change(None)

# display the widgets with the title and initial result
display(start_year_slider, end_year_slider, n_coldest_years_slider, output)
initial_result = compute_and_display_design_index(
    start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
)